In [ ]:
import cv2 as cv
import numpy as np
import torch
from torchvision import models, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load ResNet50 without classifier
resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
resnet.eval().to(device)

preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def feature_extraction(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    x = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        f = resnet(x)
    return f.view(-1).cpu().numpy()   # (2048,)


In [2]:
import os
import numpy as np
import cv2 as cv

DATASET_SRC = "../aug_dataset"

labels = {
    "glass": 0,
    "paper": 1,
    "cardboard": 2,
    "plastic": 3,
    "metal": 4,
    "trash": 5
}

X, y = [], []

for cls, label in labels.items():
    folder = os.path.join(DATASET_SRC, cls)
    for name in os.listdir(folder):
        img = cv.imread(os.path.join(folder, name))
        if img is None:
            continue
        X.append(feature_extraction(img))
        y.append(label)

X = np.array(X)   # (N, 2048)
y = np.array(y)


FEATURES_DST = "../features/"
os.makedirs(FEATURES_DST, exist_ok=True)

np.savez("../features/features_dataset.npz", X=X, y=y)
print(X.shape, y.shape)


(3000, 2048) (3000,)
